Idea 2: Using only word features


In [ ]:
import numpy as np
import os
import networkx as nx
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from numpy import dot
from numpy.linalg import norm
import random

from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd './drive/MyDrive/232EProject1'

/content/drive/MyDrive/232EProject1


In [ ]:
all_data = []
all_edges = []

for root,dirs,files in os.walk('./cora'):
    for file in files:
        if '.content' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
        elif 'cites' in file:
            with open(os.path.join(root,file),'r') as f:
                all_edges.extend(f.read().splitlines())

In [ ]:
label_encoder = {'Case_Based': 0,
                'Genetic_Algorithms': 1,
                'Neural_Networks': 2,
                'Probabilistic_Methods': 3,
                'Reinforcement_Learning': 4,
                'Rule_Learning': 5,
                'Theory': 6}

In [ ]:
#parse the data
labels = []
nodes = []
X = []
element_to_ind  = {}

for i,data in enumerate(all_data):
    elements = data.split('\t')
    labels.append(label_encoder[elements[-1]])
    X.append(elements[1:-1])
    nodes.append(elements[0])
    element_to_ind[elements[0]]= i
X = np.array(X,dtype=int)
N = X.shape[0] #the number of nodes
F = X.shape[1] #the size of node features


#parse the edge
edge_list=[]
for edge in all_edges:
    e = edge.split('\t')
    edge_list.append((e[0],e[1]))

In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)
G = nx.relabel_nodes(G, element_to_ind)
nodes = list(G.nodes)
df = pd.DataFrame(list(zip(nodes, labels,X)),columns =['node', 'label','features'])
print(len(df))
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
gcc_nodes = list(G.nodes)
df = df.loc[df['node'].isin(gcc_nodes)]
df['node'] = list(range(len(df))) #rename nodes
G = nx.relabel_nodes(G, df['node'])

2708


In [ ]:
temp = df.to_numpy()
node_idx = temp[:,0].astype(int)
y = temp[:,1].astype(int)
X = np.array([list(t) for t in temp[:,2]])

In [ ]:
def get_data(labels, cap = 20, val_count = 500, test_count = 1000):
  # to extract 20 samples for each class 
    labelCount = dict((l, 0) for l in labels)
    train_idx = []

    for i in range(len(labels)):
        label = labels[i]
        if labelCount[label]<cap:
            train_idx.append(i)
            labelCount[label]+=1
        
        #exit the loop once we found 20 examples for each class
        if all(count == cap for count in labelCount.values()):
            break  
    # remain
    rem_idx= [x for x in range(len(labels)) if x not in train_idx]
    #get the first val_num
    val_index  = rem_idx[:val_count]
    test_index = rem_idx[val_count:(val_count+test_count)]
    return train_idx, val_index,test_index

## Using only text features 

In [ ]:
train_idx,val_idx,test_idx = get_data(y)

train_mask = np.zeros((2485,),dtype=bool)
train_mask[train_idx] = True

val_mask = np.zeros((2485,),dtype=bool)
val_mask[val_idx] = True

test_mask = np.zeros((2485,),dtype=bool)
test_mask[test_idx] = True

In [ ]:
clf = SVC(kernel='rbf', gamma='auto', C=1)
clf.fit(X, y, sample_weight=train_mask)

SVC(C=1, gamma='auto')

In [ ]:
preds = clf.predict(X[val_mask])
accuracy_score(y[val_mask] , preds)

0.53

In [ ]:
preds = clf.predict(X[test_mask])
accuracy_score(y[test_mask] , preds)

0.577